<a href="https://colab.research.google.com/github/MichelleFn/datamining/blob/main/dataming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn import datasets
from sklearn import preprocessing
from scipy.stats import pearsonr
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
folder='/content/drive/My Drive/Data_Mining'
!ls '/content/drive/My Drive/Data_Mining'

Mounted at /content/drive
 datamining.ipynb      'listings_july (1).gsheet'   listings_june.csv
'listings (1).gsheet'   listings_july.csv	    listings_thes_july.csv
'listings (2).gsheet'   listings_july.gsheet	    listings_thes_june.csv


In [ ]:
# Φόρτωση δεδομένων Ιουνίου df και Ιουλίου df1
df = pd.read_csv(folder+'/listings_june.csv')
df1 = pd.read_csv(folder+'/listings_july.csv')

ΑΡΧΙΚΗ ΕΠΕΞΕΡΓΑΣΙΑ


In [ ]:
to_drop = ['host_name','host_id','name','id','neighbourhood_group','last_review']

In [ ]:
df.drop(to_drop, inplace=True, axis=1)

In [ ]:
df1.drop(to_drop, inplace=True, axis=1)

In [ ]:
mean_review = int(df['reviews_per_month'].mean())
mean_review
df['reviews_per_month'] = df['reviews_per_month'].fillna(mean_review)

In [ ]:
mean_review = int(df1['reviews_per_month'].mean())
mean_review
df1['reviews_per_month'] = df1['reviews_per_month'].fillna(mean_review)

In [ ]:
df['cost_per_stay'] = df['price'] * df['minimum_nights']

In [ ]:
df1['cost_per_stay'] = df1['price'] * df1['minimum_nights']

In [ ]:
df = df[df['minimum_nights'] <= 10]

In [ ]:
df1 = df1[df1['minimum_nights'] <= 10]

In [ ]:
df['target_price_group'] = 0
df['target_price_group'] = np.where((df['price'] > 0) & (df['price'] <= 50), 50, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 50) & (df['price'] <= 101), 100, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 101) & (df['price'] <= 201), 200, df['target_price_group'])
df['target_price_group'] = np.where((df['price'] > 200) & (df['price'] <= 500), 500, df['target_price_group'])

In [ ]:
df1['target_price_group'] = 0
df1['target_price_group'] = np.where((df1['price'] > 0) & (df1['price'] <= 50), 50, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 50) & (df1['price'] <= 101), 100, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 101) & (df1['price'] <= 201), 200, df1['target_price_group'])
df1['target_price_group'] = np.where((df1['price'] > 200) & (df1['price'] <= 500), 500, df1['target_price_group'])

In [ ]:
!pip install category_encoders

     |████████████████████████████████| 81kB 3.6MB/s 


In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
df_encoded = encoder.fit_transform(df)
df_encoded

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,neighbourhood_ΑΜΠΕΛΟΚΗΠΟΙ,neighbourhood_ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,neighbourhood_ΚΕΡΑΜΕΙΚΟΣ,neighbourhood_ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,neighbourhood_ΠΑΓΚΡΑΤΙ,neighbourhood_ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ,neighbourhood_ΣΤΑΔΙΟ,neighbourhood_ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,neighbourhood_1Ο ΝΕΚΡΟΤΑΦΕΙΟ,neighbourhood_ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,neighbourhood_ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,neighbourhood_ΓΟΥΒΑ,neighbourhood_ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,neighbourhood_ΝΕΑ ΚΥΨΕΛΗ,neighbourhood_ΒΟΤΑΝΙΚΟΣ,neighbourhood_ΑΝΩ ΠΑΤΗΣΙΑ,neighbourhood_ΚΟΛΩΝΑΚΙ,neighbourhood_ΝΕΟΣ ΚΟΣΜΟΣ,neighbourhood_ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ,neighbourhood_ΙΛΙΣΙΑ,neighbourhood_ΘΗΣΕΙΟ,neighbourhood_ΠΕΔΙΟ ΑΡΕΩΣ,neighbourhood_ΠΕΤΡΑΛΩΝΑ,neighbourhood_ΑΚΡΟΠΟΛΗ,neighbourhood_ΠΡΟΜΠΟΝΑ,neighbourhood_ΓΟΥΔΙ,neighbourhood_ΓΚΑΖΙ,neighbourhood_ΚΟΛΩΝΟΣ,neighbourhood_ΝΙΡΒΑΝΑ,neighbourhood_ΖΑΠΠΕΙΟ,neighbourhood_ΑΝΩ ΚΥΨΕΛΗ,neighbourhood_ΚΥΨΕΛΗ,neighbourhood_ΛΥΚΑΒΗΤΤΟΣ,neighbourhood_ΓΚΥΖΗ,neighbourhood_ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,neighbourhood_ΠΑΤΗΣΙΑ,neighbourhood_ΕΛΛΗΝΟΡΩΣΩΝ,neighbourhood_ΠΕΝΤΑΓΩΝΟ,neighbourhood_ΡΙΖΟΥΠΟΛΗ,neighbourhood_ΠΟΛΥΓΩΝΟ,neighbourhood_ΣΕΠΟΛΙΑ,neighbourhood_ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ,neighbourhood_ΡΗΓΙΛΛΗΣ,neighbourhood_ΚΟΛΟΚΥΝΘΟΥ,latitude,longitude,room_type_Entire home/apt,room_type_Private room,room_type_Hotel room,room_type_Shared room,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,cost_per_stay,target_price_group
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.988630,23.765270,1,0,0,0,122,1,25,0.23,6,365,122,200
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.989030,23.764480,1,0,0,0,45,1,34,0.36,6,271,45,50
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.988880,23.764730,1,0,0,0,37,1,48,0.51,6,347,37,50
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.989030,23.764480,1,0,0,0,41,1,21,0.17,6,275,41,50
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.989240,23.765000,1,0,0,0,47,1,17,0.18,6,262,47,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11309,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.981530,23.762740,1,0,0,0,44,2,0,1.00,18,0,88,50
11310,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.999300,23.724330,1,0,0,0,45,3,0,1.00,17,355,135,50
11311,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.959190,23.718510,1,0,0,0,55,3,0,1.00,17,365,165,100
11312,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.977478,23.729249,1,0,0,0,180,1,0,1.00,2,357,180,200


In [ ]:
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
df1_encoded = encoder.fit_transform(df1)
df1_encoded

/usr/local/lib/python3.6/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,neighbourhood_ΑΜΠΕΛΟΚΗΠΟΙ,neighbourhood_ΕΜΠΟΡΙΚΟ ΤΡΙΓΩΝΟ-ΠΛΑΚΑ,neighbourhood_ΚΕΡΑΜΕΙΚΟΣ,neighbourhood_ΑΓΙΟΣ ΝΙΚΟΛΑΟΣ,neighbourhood_ΠΛΑΤΕΙΑ ΑΤΤΙΚΗΣ,neighbourhood_ΣΤΑΔΙΟ,neighbourhood_ΑΓΙΟΣ ΚΩΝΣΤΑΝΤΙΝΟΣ-ΠΛΑΤΕΙΑ ΒΑΘΗΣ,neighbourhood_1Ο ΝΕΚΡΟΤΑΦΕΙΟ,neighbourhood_ΑΚΑΔΗΜΙΑ ΠΛΑΤΩΝΟΣ,neighbourhood_ΚΟΥΚΑΚΙ-ΜΑΚΡΥΓΙΑΝΝΗ,neighbourhood_ΓΟΥΒΑ,neighbourhood_ΜΟΥΣΕΙΟ-ΕΞΑΡΧΕΙΑ-ΝΕΑΠΟΛΗ,neighbourhood_ΝΕΑ ΚΥΨΕΛΗ,neighbourhood_ΠΑΓΚΡΑΤΙ,neighbourhood_ΒΟΤΑΝΙΚΟΣ,neighbourhood_ΑΝΩ ΠΑΤΗΣΙΑ,neighbourhood_ΝΕΟΣ ΚΟΣΜΟΣ,neighbourhood_ΚΟΛΩΝΑΚΙ,neighbourhood_ΠΛΑΤΕΙΑ ΑΜΕΡΙΚΗΣ,neighbourhood_ΙΛΙΣΙΑ,neighbourhood_ΑΚΡΟΠΟΛΗ,neighbourhood_ΘΗΣΕΙΟ,neighbourhood_ΠΕΔΙΟ ΑΡΕΩΣ,neighbourhood_ΠΕΤΡΑΛΩΝΑ,neighbourhood_ΠΡΟΜΠΟΝΑ,neighbourhood_ΓΚΑΖΙ,neighbourhood_ΚΟΛΩΝΟΣ,neighbourhood_ΝΙΡΒΑΝΑ,neighbourhood_ΖΑΠΠΕΙΟ,neighbourhood_ΑΝΩ ΚΥΨΕΛΗ,neighbourhood_ΚΥΨΕΛΗ,neighbourhood_ΛΥΚΑΒΗΤΤΟΣ,neighbourhood_ΓΚΥΖΗ,neighbourhood_ΑΓΙΟΣ ΕΛΕΥΘΕΡΙΟΣ,neighbourhood_ΠΑΤΗΣΙΑ,neighbourhood_ΕΛΛΗΝΟΡΩΣΩΝ,neighbourhood_ΠΕΝΤΑΓΩΝΟ,neighbourhood_ΡΙΖΟΥΠΟΛΗ,neighbourhood_ΠΟΛΥΓΩΝΟ,neighbourhood_ΓΟΥΔΙ,neighbourhood_ΣΕΠΟΛΙΑ,neighbourhood_ΣΤΑΘΜΟΣ ΛΑΡΙΣΗΣ,neighbourhood_ΡΗΓΙΛΛΗΣ,neighbourhood_ΚΟΛΟΚΥΝΘΟΥ,latitude,longitude,room_type_Entire home/apt,room_type_Private room,room_type_Hotel room,room_type_Shared room,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,cost_per_stay,target_price_group
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.98863,23.765270,1,0,0,0,63,1,25,0.22,6,356,63,100
1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.98903,23.764480,1,0,0,0,39,1,36,0.37,6,246,39,50
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.98888,23.764730,1,0,0,0,39,1,48,0.50,6,345,39,50
3,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.98903,23.764480,1,0,0,0,31,1,21,0.17,6,227,31,50
4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.98924,23.765000,1,0,0,0,34,1,17,0.18,6,237,34,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11232,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.96450,23.755190,1,0,0,0,36,1,0,1.00,2,0,36,50
11233,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.97815,23.709680,1,0,0,0,34,2,0,1.00,5,331,68,50
11234,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,38.01984,23.732670,1,0,0,0,50,1,0,1.00,2,0,50,50
11236,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,37.96640,23.746534,1,0,0,0,40,2,0,1.00,1,169,80,50


In [ ]:
df_encoded[df_encoded['price'] > 500].groupby('price')['price'].count()
df_outliers = df_encoded[df_encoded['price'] < 500]

In [ ]:
df1_encoded[df1_encoded['price'] > 500].groupby('price')['price'].count()
df1_outliers = df_encoded[df_encoded['price'] < 500]

ΜΕΘΟΔΟΙ REGRESSION

In [ ]:
#linear regression model for july data
def LinearRegression(x_train,y_train,x_test,y_test):
 from sklearn.linear_model import LinearRegression

 linear_reg_model = LinearRegression().fit(x_train, y_train)
 y_pred = linear_reg_model.predict(x_test)  

 print("intercept: " ,linear_reg_model.intercept_)
 print("coefficceiance :", linear_reg_model.coef_)

 print("r2_score: ",r2_score(y_test, y_pred))
 print("mean_squared_error: " ,mean_squared_error(y_test,y_pred))
# print(y_test)
# print(y_pred)

In [ ]:
#decision tree regression for july data
def DecisionTreeRegression(x_train,y_train,x_test,y_test):
 from sklearn.tree import DecisionTreeRegressor

 d_tree_reg_model = DecisionTreeRegressor()
 d_tree_reg_model.fit(x_train,y_train)
 y_pred_tree = d_tree_reg_model.predict(x_test)
 print(y_test)
 print(y_pred_tree)

 print("mean squared error: ",mean_squared_error(y_test,y_pred_tree))
 print("r2_score:" ,r2_score(y_test,y_pred_tree))

In [ ]:
#random forest regression for july data
def RandomForestRegression(x_train,y_train,x_test,y_test):
  from sklearn.ensemble import RandomForestRegressor

  random_forest_reg_model = RandomForestRegressor(n_estimators=200, oob_score=True, random_state=0)
  random_forest_reg_model.fit(x_train, y_train)
  y_pred_rf = random_forest_reg_model.predict(x_test)
  print(y_pred_rf)
  print(y_test)
  print("mean squared error: ",mean_squared_error(y_test,y_pred_rf))
  print("r2_score:" ,r2_score(y_test,y_pred_rf))

In [ ]:
#regression with svr for july data
def SVRegression(x_train,y_train,x_test,y_test):
  from sklearn.svm import SVR
  svr_model = SVR()
  svr_model.fit(x_train,y_train)
  y_pred_svr = svr_model.predict(x_test)
  print(y_pred_svr)
  print(y_test)

  print("mean squared error: ",mean_squared_error(y_test,y_pred_svr))
  print("r2_score:" ,r2_score(y_test,y_pred_svr))

In [ ]:
from sklearn.model_selection import train_test_split
x_alt_out = df_outliers.loc[:, df_outliers.columns != 'price']
y_alt_out  = df_outliers[["price"]]
x_train_out, x_test_out, y_train_out, y_test_out = train_test_split(x_alt_out, y_alt_out, train_size=0.9, random_state=None)

In [ ]:
LinearRegression(x_train_out,y_train_out,x_test_out,y_test_out)

intercept:  [-3.49066931e+10]
coefficceiance : [[ 3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
   3.49066938e+10  3.49066938e+10  3.49066938e+10  3.49066938e+10
  -7.94692323e+01  9.74303818e+01  4.10564160e+00  1.51363373e-01
   1.08094062e+01 -1.50667607e+01 -8.13148427e+00 -1.22823715e-02
  -6.28020287e-01  3.65865231e-02  3.16882133e-03  1.53294325e-01
   4.23978567e-01]]
r2_score:

In [ ]:
DecisionTreeRegression(x_train_out,y_train_out,x_test_out,y_test_out)

      price
9150     75
4659     70
514     261
4502    250
4214     35
...     ...
1821    120
3671     25
476      48
6488    200
4310     25

[1092 rows x 1 columns]
[ 75.  70. 300. ...  48. 200.  25.]
mean squared error:  17.156593406593405
r2_score: 0.9931215572233751


In [ ]:
RandomForestRegression(x_train_out,y_train_out,x_test_out,y_test_out)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[ 75.    70.   308.78 ...  48.   199.84  25.  ]
      price
9150     75
4659     70
514     261
4502    250
4214     35
...     ...
1821    120
3671     25
476      48
6488    200
4310     25

[1092 rows x 1 columns]
mean squared error:  20.9341608058608
r2_score: 0.9916070502012108


In [ ]:
SVRegression(x_train_out,y_train_out,x_test_out,y_test_out)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[ 75.49115308  57.43580728 108.21280738 ...  42.07615192 124.63988653
  38.42246063]
      price
9150     75
4659     70
514     261
4502    250
4214     35
...     ...
1821    120
3671     25
476      48
6488    200
4310     25

[1092 rows x 1 columns]
mean squared error:  599.3612304311092
r2_score: 0.7597033497067389


In [ ]:
from sklearn.model_selection import train_test_split
x_alt_out_july = df1_outliers.loc[:, df1_outliers.columns != 'price']
y_alt_out_july  = df1_outliers[["price"]]
x_train_out_july, x_test_out_july, y_train_out_july, y_test_out_july = train_test_split(x_alt_out_july, y_alt_out_july, train_size=0.9, random_state=None)

In [ ]:
LinearRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

intercept:  [3.25221838e+11]
coefficceiance : [[-3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -3.25221836e+11 -3.25221836e+11 -3.25221836e+11 -3.25221836e+11
  -1.10604362e+02  9.61493530e+01  4.53991318e+00  1.01015472e+00
   1.02380428e+01 -1.58117995e+01 -7.73149490e+00 -1.26647949e-02
  -6.53666973e-01  3.16886902e-02  3.36074829e-03  1.41173363e-01
   4.34730530e-01]]
r2_score: 

In [ ]:
DecisionTreeRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

       price
5642      18
10433     60
5722      30
6602      40
7135      70
...      ...
10166     23
1283     111
8830      37
7451      66
10890     90

[1092 rows x 1 columns]
[18. 60. 30. ... 37. 66. 90.]
mean squared error:  20.727106227106226
r2_score: 0.9908371397281618


In [ ]:
RandomForestRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


[17.165 60.    30.01  ... 37.    66.    90.   ]
       price
5642      18
10433     60
5722      30
6602      40
7135      70
...      ...
10166     23
1283     111
8830      37
7451      66
10890     90

[1092 rows x 1 columns]
mean squared error:  3.801901121794874
r2_score: 0.9983192883577354


In [ ]:
SVRegression(x_train_out_july,y_train_out_july,x_test_out_july,y_test_out_july)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[34.10904701 70.39994309 36.63164938 ... 35.52075531 59.5997954
 66.50756646]
       price
5642      18
10433     60
5722      30
6602      40
7135      70
...      ...
10166     23
1283     111
8830      37
7451      66
10890     90

[1092 rows x 1 columns]
mean squared error:  427.9779214602129
r2_score: 0.8108032133958343


ΜΕΘΟΔΟΙ ΚΑΤΗΓΟΡΙΟΠΟΙΗΣΗΣ

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

In [ ]:
#decision trees classifier
def DecisionTreeClassifier(x_train,y_train,x_test,y_test):
 from sklearn.tree import DecisionTreeClassifier
 dec_tree_clas = DecisionTreeClassifier(criterion='entropy',random_state=0) 
 dec_tree_clas.fit(x_train,y_train)
 y_pred_dec_tree_clas = dec_tree_clas.predict(x_test)
 percision = precision_score(y_test, y_pred_dec_tree_clas,average=None)
 recall = recall_score(y_test, y_pred_dec_tree_clas,average=None)
 f1 = f1_score(y_test, y_pred_dec_tree_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)

#  print(confusion_matrix(y_test_class, y_pred_dec_tree_clas))
#  print(classification_report(y_test_class, y_pred_dec_tree_clas))
 print(accuracy_score(y_test, y_pred_dec_tree_clas))

In [ ]:
#random forest classification
def RandomForestClassification(x_train,y_train,x_test,y_test):
 from sklearn.ensemble import RandomForestClassifier

# Create the model with 100 trees
 random_forest_class_model = RandomForestClassifier(n_estimators=100, 
                               bootstrap = True,
                               max_features = 'sqrt')
# Fit on training data
 random_forest_class_model.fit(x_train, y_train)
 y_pred_rf_clas = random_forest_class_model.predict(x_test)
 percision = precision_score(y_test,y_pred_rf_clas,average=None)
 recall = recall_score(y_test, y_pred_rf_clas,average=None)
 f1 = f1_score(y_test,y_pred_rf_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)
#  print(confusion_matrix(y_test_class, y_pred_rf_clas))
#  print(classification_report(y_test_class, y_pred_rf_clas))
 print(accuracy_score(y_test, y_pred_rf_clas))


In [ ]:
def SVClassification(x_train,y_train,x_test,y_test):
 from sklearn.svm import SVC
 SupportVectorClassModel = SVC()
 SupportVectorClassModel.fit(x_train,y_train)
 y_pred_sv_clas = SupportVectorClassModel.predict(x_test)

 percision = precision_score(y_test, y_pred_sv_clas,average=None)
 recall = recall_score(y_test,y_pred_sv_clas,average=None)
 f1 = f1_score(y_test, y_pred_sv_clas,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)

#  print(confusion_matrix(y_test_class, y_pred_sv_clas))
#  print(classification_report(y_test_class, y_pred_sv_clas))
 print(accuracy_score(y_test, y_pred_sv_clas))


In [ ]:
#knn classification
def knnClassification(x_train,y_train,x_test,y_test):
 from sklearn.preprocessing import StandardScaler
 scaler = StandardScaler()
 scaler.fit(x_train)

 x_train_scaled = scaler.transform(x_train)
 x_test_scaled = scaler.transform(x_test)

 from sklearn.neighbors import KNeighborsClassifier
 classifier = KNeighborsClassifier(n_neighbors=5)
 classifier.fit(x_train_scaled, y_train)
 y_pred_knn = classifier.predict(x_test_scaled)

 from sklearn.metrics import classification_report, confusion_matrix

 percision = precision_score(y_test, y_pred_knn,average=None)
 recall = recall_score(y_test, y_pred_knn,average=None)
 f1 = f1_score(y_test, y_pred_knn,average=None)

 print("percision:", percision)
 print("recall:", recall)
 print("f1_score:", f1)
#  print(confusion_matrix(y_test_class, y_pred_knn))
#  print(classification_report(y_test_class, y_pred_knn))
 print(accuracy_score(y_test, y_pred_knn))


In [ ]:
from sklearn.model_selection import train_test_split
x_alt_class = df_outliers.loc[:, df_outliers.columns != 'target_price_group']
y_alt_class  = df_outliers[["target_price_group"]]
x_train_alt_class, x_test_alt_class, y_train_alt_class, y_test_alt_class = train_test_split(x_alt_class, y_alt_class, train_size=0.9, random_state=None)

In [ ]:
RandomForestClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


percision: [1.         0.99712644 1.         1.        ]
recall: [1.         1.         0.98958333 1.        ]
f1_score: [1.         0.99856115 0.9947644  1.        ]
0.9990842490842491


In [ ]:
knnClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if sys.path[0] == '':


percision: [0.86127168 0.79047619 0.90909091 0.84210526]
recall: [0.94904459 0.71757925 0.625      0.76190476]
f1_score: [0.9030303  0.75226586 0.74074074 0.8       ]
0.8434065934065934


In [ ]:
DecisionTreeClassifier(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

percision: [1. 1. 1. 1.]
recall: [1. 1. 1. 1.]
f1_score: [1. 1. 1. 1.]
1.0


In [ ]:
SVClassification(x_train_alt_class,y_train_alt_class,x_test_alt_class,y_test_alt_class)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


percision: [0.97364341 0.97345133 1.         1.        ]
recall: [1.         0.95100865 0.90625    1.        ]
f1_score: [0.98664572 0.96209913 0.95081967 1.        ]
0.9761904761904762


In [ ]:
from sklearn.model_selection import train_test_split
x_alt_class_july = df1_outliers.loc[:, df1_outliers.columns != 'target_price_group']
y_alt_class_july  = df1_outliers[["target_price_group"]]
x_train_alt_class_july, x_test_alt_class_july, y_train_alt_class_july, y_test_alt_class_july = train_test_split(x_alt_class_july, y_alt_class_july, train_size=0.9, random_state=None)

In [ ]:
knnClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  if sys.path[0] == '':


percision: [0.88550725 0.79220779 0.85333333 1.        ]
recall: [0.95023328 0.75076923 0.66666667 0.67857143]
f1_score: [0.91672918 0.77093207 0.74853801 0.80851064]
0.8589743589743589


In [ ]:
DecisionTreeClassifier(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

percision: [1. 1. 1. 1.]
recall: [1. 1. 1. 1.]
f1_score: [1. 1. 1. 1.]
1.0


In [ ]:
SVClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


percision: [0.98018293 0.96296296 0.97647059 0.96296296]
recall: [1.         0.96       0.86458333 0.92857143]
f1_score: [0.9899923  0.9614792  0.91712707 0.94545455]
0.9743589743589743


In [ ]:
RandomForestClassification(x_train_alt_class_july,y_train_alt_class_july,x_test_alt_class_july,y_test_alt_class_july)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  # Remove the CWD from sys.path while we load stuff.


percision: [1.         0.99693252 0.97938144 1.        ]
recall: [1.         1.         0.98958333 0.92857143]
f1_score: [1.         0.9984639  0.98445596 0.96296296]
0.9972527472527473
